In [1]:
!nvidia-smi

Mon Dec  2 02:23:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              42W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
#BASE_DIR= '/content/drive/MyDrive/DACON/Finance/reprocessed/'
BASE_DIR='/content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/'

# 설명

## Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

## Mount/Login

구글 드라이브를 마운트하고 허깅페이스에 로그인
- 이때 허깅페이스 토큰은 kdt3 그룹에 대해 읽기/쓰기 권한이 있는 토큰이어야 함

## Download Library
필요/사용 라이브러리 다운로드
이때 버전 문제로 설치를 한 뒤 세션을 한번 재시작해줘야 합니다
<br>(그리고 세션 완전히 끊기면 다운로드 후 재시작을 다시 해줘야...)

## Import Library
한번 재시작했으면 위 과정 없이 Import만 실행해주면 됩니다

## Vector DB
문서를 여러 조각(chunk)로 나누고, 임베딩 유사도를 통해 관련 조각을 찾을 수 있게 DB화하는 함수들이 정의되어 있습니다.

## DB 생성
Vector DB에서 정의된 함수들로 문서 DB를 만들어줍니다.<br><br>
이때 Train과 Test를 한번에 하려고 하면 코랩이 터질 확률이 높으므로 Train하고 Create Dataset까지 실행해 업로드 한 뒤 재시작해서 램을 비우고 Test를 하는 것이 좋습니다.<br> 또한 문서 임베딩을 어떤 모델로 할지 인자로 넘겨줄 수 있습니다

## Create Dataset
DB 생성에서 만든 db와 데이터 dataframe을 사용해 HuggingFace 데이터셋 생성 후 업로드

## Fine-Tuning
학습 데이터셋으로 모델에 대한 파인튜닝 진행 후 Huggingface에 업로드<br>
4비트 양자화 LoRA로 파인튜닝<br>
기반 모델 또는 넣어줄때 사용할 프롬프트, 학습 관련 하이퍼파라미터 수정 가능

## Langchain 을 이용한 추론
모델을 사용한 추론


## 실행
### 기본
Mount/Login -> Download Library -> 재시작 (처음 1번)
Mount/Login -> Import Library (이후)

### 데이터셋 만들기
기본 -> Vector DB -> DB 생성 -> Create Dataset에서 첫 셀 + Train/Valid/Test 중 해당하는 셀

### 모델 학습하기
기본 -> Fine-Tuning(업로드할 위치, 데이터셋 위치, 모델 링크 확인 필수)

### 학습된 모델로 추론하기
기본 -> Langchain을 이용한 추론(모델 링크, 데이터셋 위치 확인) -> Submission(저장할 파일명 확인)

# Mount/Login

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
ls {BASE_DIR}

241008_csv_checker.ipynb             gemma2_financeQA-finetune/  test_source/
combined_train_aug_v3.5_editted.csv  processed/                  train.csv
combined_train_aug_v3.csv            sample_submission.csv       train_source/
combined_train_aug_v3_editted.csv    sub/                        Untitled0.ipynb
data/                                temp/
eval/                                test.csv


In [5]:
import os

token_path = os.path.join(BASE_DIR,'data','token')
with open(token_path,'r') as f:
    master_token = f.readline().strip('\n')

In [6]:
from huggingface_hub import login

login(token=master_token, add_to_git_credential=True)

# Download Library

In [7]:
!apt-get install tesseract-ocr
!apt-get install poppler-utils

!pip install orjson==3.10.6

!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install langchain-teddynote
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu
!pip install unstructured pdfminer.six
!pip install pillow-heif
!pip install pikepdf pypdf

!pip install pymupdf4llm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple/


# Import Library

In [8]:
import os
import unicodedata
import torch
import pandas as pd
from tqdm.auto import tqdm
import fitz  # PyMuPDF

from langchain.document_loaders.parsers.pdf import PDFPlumberParser

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter

# PDF 로딩/청크화 관련
from langchain.document_loaders.parsers.pdf import PDFPlumberParser
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.document_loaders import UnstructuredPDFLoader
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever

from unstructured.cleaners.core import clean_extra_whitespace, clean, clean_non_ascii_chars

import pymupdf4llm
import pymupdf

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [9]:
# gpu memory 할당 해제
import gc, time

def free_cuda():
  mem = 1
  while mem > 0 :
    time.sleep(0.5)
    mem = gc.collect()
    torch.cuda.empty_cache()
    print("freed : ",mem)

# Vector DB

In [10]:
from operator import itemgetter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from unstructured.cleaners.core import clean_extra_whitespace, clean, clean_non_ascii_chars


# 불릿포인트 제거용 함수
def remove_bulletpoints(text):
    cleaned_text = text
    for symbol in ['ㅇ','-','□', '※', '▸','∙','●','☞','■','','','·']:
        cleaned_text = cleaned_text.replace(symbol, f"-")
    return cleaned_text

def replace_sign_symbol(text):
    cleaned_text = text
    cleaned_text = cleaned_text.replace('△', "-")
    return cleaned_text


# 숫자 심볼 숫자로 변환
def replace_num_symbols_with_number(text):
    cleaned_text = text
    for idx, symbol in enumerate(['①', '②', '③', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '⑪', '⑫', '⑬', '⑭', '⑮']):
        cleaned_text = cleaned_text.replace(symbol, f"{idx+1})")
    return cleaned_text

def erase_unicode_chr(text):
  return re.sub(r'\\u[0-9a-fA-F]{4}','-',text)

In [11]:
def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)

def process_path(base_dir,file_path):
  norm_path = normalize_path(file_path)
  if not os.path.isabs(norm_path):
    return os.path.normpath(os.path.join(base_dir, norm_path))
  else : return norm_path

def subpath_list(dir_path):
  return list(map(lambda x : os.path.join(dir_path,x),os.listdir(dir_path)))

def processed_path_matcher(dir_path,file_path):
  sub_list = subpath_list(dir_path)
  path_list = list()
  for sub in sub_list:
    path_list.extend(subpath_list(sub))
  prcssd_list =list(map(normalize_path,path_list))
  for real_path,prcssd_path in zip(path_list,prcssd_list) :
    if file_path == prcssd_path : return real_path
  else : return file_path

In [12]:
from operator import itemgetter
import re

def remove_table_spaces(text):
  text = re.sub(r'[ \t\r]+',' ',text)
  text = re.sub(r'[\n\v\f]+','\n',text)
  text = re.sub(r'\|:?[\-]+:?(?=[\|])','|-',text)
  return text


def clean_string(text):
    text_string = clean(text, dashes=True,trailing_punctuation=True, bullets=True)
    text_string = replace_num_symbols_with_number(text_string)
    text_string = remove_bulletpoints(text_string)
    return text_string

def clean_table(text_string):
#    text_string = remove_table_spaces(text_string)
    text_string = replace_num_symbols_with_number(text_string)
    text_string = replace_sign_symbol(text_string)
    text_string = remove_bulletpoints(text_string)
    return erase_unicode_chr(text_string)

# 전체 마크다운 처리
def process_pdf(file_path, chunk_size=256, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = pymupdf4llm.to_markdown(file_path)

    headers_to_split_on = [
        ("#","Header 1"),
        ("##","Header 2"),
        ("###","Header 3"),
    ]

    md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_splitter.split_text(doc)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(md_chunks)

    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db




In [13]:
import pickle

def check_and_mkdir(func):
    def wrapper(*args,**kwargs):
        if not os.path.exists(args[0]): os.makedirs(args[0])
        return func(*args,**kwargs)
    return wrapper

@check_and_mkdir
def save_pkl(save_dir,file_name,save_object):
    if not os.path.exists(save_dir): os.mkdir(save_dir)
    file_path = os.path.join(save_dir,file_name)
    with open(file_path,'wb') as f:
        pickle.dump(save_object,f)

def load_pkl(file_path):
    with open(file_path,'rb') as f:
        data = pickle.load(f)
    return data

# Preprocessing Tables

In [20]:
!pip install gmft
!pip install git+https://github.com/conjuncts/gmft_pymupdf.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.6 MB/s eta 0:00:00
  Cloning https://github.com/conjuncts/gmft_pymupdf.git to /tmp/pip-req-build-kh8krizb
  Running command git clone --filter=blob:none --quiet https://github.com/conjuncts/gmft_pymupdf.git /tmp/pip-req-build-kh8krizb
  Resolved https://github.com/conjuncts/gmft_pymupdf.git to commit ec63abcd8a6fd0564df765f72bc1e0009e8a5fdc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gmft_pymupdf: filename=gmft_pymupdf-0.0.1-py3-none-any.whl size=14799 sha256=49e6537a1b95dae321f6c01d69066c093280f2f2f2d0a401bf43b05fa2eff7e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-ph8ksg0r/wheels/51/c5/90/0d40554960f02c258dda3fee906d2cbc776bc371eefe4c3d9e
Successfully built

In [21]:
import gmft.table_detection
import gmft
import markdown
from gmft.auto import CroppedTable, TableDetector, AutoTableFormatter, AutoFormatConfig
from gmft.auto import AutoTableDetector, TATRDetectorConfig
from gmft.pdf_bindings import PyPDFium2Document
from gmft_pymupdf import PyMuPDFPage
from collections import defaultdict
import copy

In [22]:
!pip install ipdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.6 MB/s eta 0:00:00


In [23]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
from ipdb import set_trace

import matplotlib.pyplot as plt

def close_event():
    plt.close() #timer calls this function after 3 seconds and closes the window

fig = plt.figure()
timer = fig.canvas.new_timer(interval = 3000) #creating a timer object and setting an interval of 3000 milliseconds
timer.add_callback(close_event)

<function __main__.close_event()>

<Figure size 640x480 with 0 Axes>

## functions

### process info about page and box

In [24]:
'''
-height
8pt : 2.82mm , a4 : 210mm * 297mm
ratio : 2.82/297 ~ 0.0094
-width
same length as height : get height and use it
'''

def bound_page(box,page):
  x0 = min(max(box[0],page[0]),page[2])
  x1 = min(max(box[1],page[1]),page[3])
  x2 = max(min(box[2],page[2]),page[0])
  x3 = max(min(box[3],page[3]),page[1])
  return (x0,x1,x2,x3)

def check_exclusv_range(ran1,ran2,ths):
  flag1 = (ran1[0] - ran2[1] >= -ths)
  flag2 = (ran2[0] - ran1[1] >= -ths)
  return flag1 or flag2

def check_exclusive_box(box1,box2,ths):
  flag_horiz = check_exclusv_range((box1[0],box1[2]),(box2[0],box2[2]),ths)
  flag_verti = check_exclusv_range((box1[1],box1[3]),(box2[1],box2[3]),ths)
  return flag_horiz or flag_verti

def union_box(box1,box2):
  return min(box1[0],box2[0]),min(box1[1],box2[1]),max(box1[2],box2[2]),max(box1[3],box2[3])

def check_pairly_linked(ele_list,check_not_link,union_func):
  elements = copy.deepcopy(ele_list)
  for i0,e0 in enumerate(elements):
    for i1,e1 in enumerate(elements):
      if i0 >= i1 : continue
      if not check_not_link(e0,e1):
        elements.pop(i1)
        elements.pop(i0)
        new = union_func(e0,e1)
        elements.append(new)
    #    print(i0,i1,new,len(elements))
    #  else : print('=',i0,i1)
  return elements

import copy

def union_pairly_linked(ele_list,check_not_link,union_func):
  elements = ele_list
  cnt,bnd = 0,2**len(ele_list)
  while True :
    #print('-'*5,cnt,'-'*5)
    rslt = check_pairly_linked(elements,check_not_link,union_func)
    if len(rslt) == len(elements) or len(rslt) < 2 : break
    if cnt > bnd : break
    elements,cnt = rslt, cnt+1
  return rslt

def get_ths(page,ths_ratio=0.0094):
  return ths_ratio * (page[3]-page[1])

def organize_box(box_list,page,ths):
  box_list = list(map(lambda x : bound_page(x,page),box_list))
  check_exclusv = lambda x,y : check_exclusive_box(x,y,ths)
  return union_pairly_linked(box_list,check_exclusv,union_box)

def get_bbox(tables):
  return list(map(lambda x : x.bbox,list(tables)))

def get_page_size(page,ths=0):
  area = (0,0,page[2]-page[0],page[3]-page[1])
  return expand_bbox_by_ths(area,area,ths)

def expand_bbox_by_ths(bbox,area,ths=0):
  bbox =(bbox[0]-ths,bbox[1]-ths,bbox[2]+ths,bbox[3]+ths)
  return bound_page(bbox,area)

def larger_v_ths(area,ths=0):
  size = get_page_size(area)
  return (size[2]>=ths) and (size[3]>=ths)

def infer_bbox_pos(area,bbox):
  return bbox[0]+area[0],bbox[1]+area[1],bbox[2]+area[0],bbox[3]+area[1]

### process detector,formatter,tables

In [26]:
from gmft.auto import AutoTableDetector
from gmft_pymupdf import PyMuPDFPage


def get_ft_bbox(ft,area,ths=0):
#  return ft.rect.bbox
  rslt = organize_box(ft.fctn_results['boxes'],area,ths)[0]
  return infer_bbox_pos(ft.rect.bbox,rslt) #expand_bbox_by_ths(rslt,area,ths)

def define_formatter():
    config = AutoFormatConfig()
    config.semantic_spanning_cells=True
    config.semantic_hierarchical_left_fill='deep'
    config.enable_multi_header=False
    config.torch_device= device
    config.total_overlap_reject_threshold = 0.3
    config.large_table_assumption = True
    config.verbose = 2
    formatter = AutoTableFormatter(config=config)
    return formatter

def define_detector():
    config =TATRDetectorConfig()
    config.torch_device= device
    config.detector_base_threshold=0.75
#    config.detector_base_threshold=0.6
    detector = AutoTableDetector(config=config)
    return detector
import re

def erase_constant_rowcol(df,val):
  cols = range(len(df.columns))
  df_temp = df.set_axis(cols,axis=1)
  cond = df_temp == val
  target = list(filter(lambda col : np.sum(cond[col]) != len(cond),cols))
  df_temp, cond = df_temp[target], cond[target]
  cond2 = np.sum(cond,axis=1) != len(target)
  return df_temp[cond2]

def maybe_numeric_table(df,ths=0.35,line_ths=0.9):
  df_temp = df.replace(to_replace=[None], value=0).fillna('').astype(str)
  df_temp = erase_constant_rowcol(df_temp,'0')
  #df_temp = df.replace(r'(?:(\d+?)),(\d+?)',r'\1\2',regex=True)
  df_temp = df.replace(r'[△\,\(\)\-\+\.\s\%\[\]]','',regex=True)
  df_temp = df_temp[list(df_temp.columns)].apply(pd.to_numeric,errors='coerce')
  rslt = ~df_temp.isna()
  rowwise = rslt.apply(sum,axis=1)
  colwise = rslt.apply(sum,axis=0)
  if np.sum(rowwise * line_ths <= len(rslt)) > 0 : return True
  if np.sum(colwise * line_ths <= len(rslt.columns)) > 0 : return True
  if np.sum(rslt.values) > len(rslt)*len(rslt.columns)*ths : return True
  return False

def check_table_df_soundness(df,ths=0.5):
  if len(df) < 1 : return False
  if maybe_numeric_table(df) : return True
  df_temp = df.replace(to_replace=[None], value='PD_NONE')
  cols = list(df.columns.astype(np.string_))
  cols = list(map(lambda x : '' if x is None else str(x),cols))
  cols = list(map(lambda x : re.sub(r'[\s]*','',x),cols))
  null_col = list(filter(lambda x: len(x)<1,cols))
  if len(null_col) > len(cols) * ths : return False
  if np.sum(df_temp=='PD_NONE') > len(df)*len(cols)*ths : return False
  return True

def detect_sound_table(dt):
  if len(dt.text()) > 2 : return True
  return False

def find_tables(page,area,ths=0):
  detector,formatter = define_detector(),define_formatter()
  doc = PyMuPDFPage(page)
  dt_whole = detector.extract(doc)
  dt_whole = list(filter(detect_sound_table,dt_whole))
  gmft_bboxes = list(map(lambda x : x.bbox,dt_whole))
  searched = get_bbox(page.find_tables())
  searched.extend(gmft_bboxes)
  searched = organize_box(searched,area,ths)
  searched = list(map(lambda x: expand_bbox_by_ths(x,area,ths),searched))
#  rslt = list(map(lambda x:make_table(x,doc,area,formatter,ths),searched))
  return list(filter(lambda x : x is not None,searched))

def make_table(bbox,doc,area,formatter,ths=0):
  rect = gmft.common.Rect(bbox)
  temp = gmft.table_detection.CroppedTable(doc,rect,0.8) #confidence level 조정이 표 인식에 영향 있을지도
  ft = formatter.extract(temp)
  #display(ft.rect.bbox,ft.visualize())
  try :
    tab_box = get_ft_bbox(ft,area,ths)
    caption = '\t'.join(ft.captions()) if 'captions' in ft.__dir__() else ''
    df_tab = ft.df()
    if not check_table_df_soundness(df_tab) :
      print('table does not sound')
      raise Exception('table does not sound')
    #else : print('table sounds')
    table = {'content':df_tab,'bbox':tab_box,'caption':caption} #, 'ft':ft}
    return table
  except Exception as e:
    try :
      display(df_tab)
      df_tab.msg = e
      table = df_tab
    except : table = None
#    display(ft.visualize())
    print('exception : ',e)
    return table

### search in page

In [27]:
'''
if table found, search left and right
and then, the area will be colored
'''

def get_area(area,page,left):
  if left : return page[0],area[1],area[0],area[3]
  else : return area[2],area[1],page[2],area[3]


def get_blank(searched_row,page,ths=0):
  rslt,point = list(), page[1]
  searched_row = sorted(searched_row,key=lambda x: x[1])
  searched_row.append((page[0],page[3],page[2],page[3]))
  for row in searched_row[:-1]:
    if row[1] - point < ths : continue
    rslt.append((page[0],point,page[2],row[1]))
    point=row[3]
  return [bound_page(box,page) for box in rslt]


def extend_list_dict(a,b):
  rslt = defaultdict(list)
  for key,val in a.items():
    rslt[key].extend(val)
  for key,val in b.items():
    rslt[key].extend(val)
  return rslt

def search_page(page,area,ths=0,depth=0):
  if not larger_v_ths(area,ths) : return list()
  if depth >=10 : raise Exception(depth)
#  print('depth : ',depth,'area : ',area)
  page.set_cropbox(area) #area : page에서 절대적 위치. cropbox를 하게 되면 상대적 위치로 바뀜
  rslt,searched = list(),list()
#  set_trace()
  detected = find_tables(page,get_page_size(area),ths)
  if len(detected)==0 : return rslt
  for target in detected:
#    if target is None : continue
#    if type(target) is not dict : rslt['errs'].append(target)
    rect = infer_bbox_pos(area,target)
    bbox = {'bbox':rect,'depth':depth}
    rslt.append(bbox)
    print('detected:',rect,'\t at',area,f' in depth {depth}')

    left_area = get_area(rect,area,True)
    right_area = get_area(rect,area,False)
    if depth > 5 : print(f'left {left_area}\tright{right_area}')
    left = search_page(page,left_area,ths,depth+1)
    right = search_page(page,right_area,ths,depth+1)
    searched.append((area[0],rect[1],area[2],rect[3]))
    if depth > 5 : print(searched)
    rslt.extend(left+right)

  if len(rslt)==0 : return rslt
  searched = organize_box(searched,area,ths)
  blanks = get_blank(searched,area,ths)
  if depth >5 : print(f'in {area}\n\t',blanks)
  for row in blanks:
    detected = search_page(page,row,ths,depth+1)
    rslt.extend(detected)

#  rslt= organize_box(rslt,area,ths) : can't apply directly like this
#  print('in search page, err : ',len(rslt['errs']))
  page.set_cropbox(area)
  return rslt

### extract tables and reform pdfs

In [14]:
def replace_area_to_mark(mu_page,area,mark):
    mu_page.add_redact_annot(area)
    mu_page.apply_redactions()
    mu_page.draw_rect(area,color=(.0,0,0),fill=(.99,.99,.99))
    rc = mu_page.insert_htmlbox(area,mark,scale_low=0)
    return mu_page

def extract_tables_from_pdf(full_path,tab_word='[[TABLE_{0}]]'):
    pdf = pymupdf.open(full_path)
    chunks, tables_dict, cnt= list(), defaultdict(list),0
    err_dict = dict()
    if pdf is None : return None, tables_dict
    for pnum, page in enumerate(tqdm(pdf)):
      page_area = tuple(page.mediabox)
      ths = get_ths(page_area)

      detected = search_page(page,page_area,ths)
      bboxes = [dic['bbox'] for dic in detected]
      bboxes = organize_box(bboxes,page_area,ths)
      tables,errors,doc =list(),list(), PyMuPDFPage(page)
      formatter = define_formatter()
      for box in bboxes:
        table = make_table(box,doc,page_area,formatter,get_ths(page_area))
        if table is None :
          print('error : ',box)
          continue
        if type(table) is not dict : errors.append(table)
        else : tables.append(table)

      if len(errors)>0 :
        err_dict[pnum] = errors
        print('errs : ',len(errors))
      if len(tables) == 0 : continue
      print(f'detected in p.{pnum} :\t',len(tables),' tables')
      tables = sorted(tables,key=lambda x : (x['bbox'][0],x['bbox'][1]))
      for idx,tab in enumerate(tables):
        tab_mark = tab_word.format(cnt+idx)
        table_md = clean_table(tab['content'].to_markdown(index=False))
        tables_dict[pnum].append((tab_mark,table_md + f"\n{tab['caption']}"))

        try :
          area = tab['bbox']
          page.add_redact_annot(area)
          page.apply_redactions()
          page.draw_rect(area,color=(.0,0,0),fill=(.99,.99,.99))
          rc = page.insert_htmlbox(area,tab_mark,scale_low=0)
        except :
          print(page.mediabox)
          print(page.cropbox)
          print(tab['bbox'])
          display(tab['content'])
          print(table_md)

      cnt+=len(tables)

    print(cnt, len(tables_dict), sum([len(tables) for tables in tables_dict.values()]))
    return pdf, tables_dict,err_dict

def extract_table_and_pdf(pdf_path,base_path,save_dir):
    # 경로 정규화 및 절대 경로 생성
    norm_path = normalize_path(pdf_path)
    full_path = process_path(base_path,pdf_path)
    full_path = processed_path_matcher(base_path,full_path)
    pdf_name = os.path.basename(full_path)

    print(f"Processing {pdf_name}...")
    save_path = os.path.join(save_dir, norm_path)
    print(full_path,save_path)
    pdf_dir = os.path.dirname(save_path)
    if not os.path.exists(pdf_dir) : os.makedirs(pdf_dir)
    new_pdf,tab_list,err_list = extract_tables_from_pdf(full_path,tab_word)
    save_pkl(pdf_dir,pdf_name[:-4]+'.pkl',tab_list)
    save_pkl(pdf_dir,'err_'+pdf_name[:-4]+'.pkl',err_list)
    new_pdf.save(save_path,garbage=4,deflate=True)
    return tab_list,err_list

def reform_pdfs_from_df(df, base_path,save_dir,name='data'):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    unique_paths = df['Source_path'].unique()
    tab_dict,err_dict = dict(), dict()
    for path in tqdm(unique_paths, desc="Processing PDFs"):
      print(path,base_path)
      tab_dict[path],err_dict[path]=extract_table_and_pdf(path,base_path,save_dir)
    save_pkl(os.path.join(save_dir,'tables'),f'tab_{name}.pkl',tab_dict)
    return tab_dict,err_dict

## read table marks

In [14]:
def convert_neg_idx(idxs,len_obj):
  rslt = deepcopy(idxs)
  for i in idxs:
    if i >= 0 : continue
    new_num = i + len_obj
    del rslt[i]
    rslt.append(new_num)
  return rslt

def add_escape(sent):
  idxs = list(filter(lambda x : sent[x] in ['[',']'],range(len(sent))))
  temp, idxs = list(sent), convert_neg_idx(idxs,len(sent))
  idxs = sorted(idxs)[::-1]
  for i in idxs:
    temp.insert(i,'\\')
  return ''.join(temp)

In [15]:
import re
from copy import deepcopy
from collections import defaultdict
from langchain_core.documents import Document as Doc

def get_former_idx(err_list):
  rslt = list()
  for i in err_list:
    cand = list(filter(lambda x : x not in err_list,range(i)))
    idx = max(cand) if cand else 0
    rslt.append(idx)
  return rslt

def get_latter_idx(err_list):
  rslt = list()
  for i in err_list:
    cand = list(filter(lambda x : x not in err_list,range(i,err_list[-1]+2)))
    idx = min(cand) if cand else err_list[-1]+1
    rslt.append(idx)
  return rslt

def make_table_page(content,tab_mark,table,tab_caption=None,th_len=100):
  if len(content)<len(tab_mark) : return None, None
  if tab_caption is None : tab_caption = tab_mark
  re_sep = '[\s\|]*'
  re_mark = insert_btwn_chr(tab_mark,re_sep)
  re_trgt = re.compile(re_mark)
  flag = list(re.finditer(re_trgt,content))
  if flag :
      front,end = flag[0].pos,flag[0].endpos
      start = min(0,front-th_len)
      new_page = content[start:front] + '\n' + table + f'\n{tab_caption}'
      #page = content[:front]+tab_caption+content[end:]
      page = content
  else : new_page, page = None, None
  return new_page,page

def get_insert_idx(former_idx,latter_idx,tab_page):
  rslt = list()
  for former,latter in zip(former_idx,latter_idx):
    pages = tab_page.values()
    first, last = 0,max(pages)
    i0 = tab_page[former] if former in tab_page else first
    i1 = tab_page[latter] if latter in tab_page else last
    rslt.append(int((i0+i1)/2))
  return rslt

def set_err_tab_page(new_pages,err_list,table_list,tab_page,tab_caption):
  if len(err_list) == 0 : return new_pages
  err_idx, err_tabs = zip(*err_list)
  if len(new_pages) == 0 : insert_idx = [-1 for _ in err_list]
  else :
    former_idx = get_former_idx(err_idx)
    latter_idx = get_latter_idx(err_idx)
    insert_idx = get_insert_idx(former_idx,latter_idx,tab_page)
  for page,i_tab,tab in zip(insert_idx,err_idx,err_tabs):
    content =tab +'\n'+ tab_caption.format(i_tab)
    new_pages[page].append(Doc(page_content=content))
  return new_pages

def convert_neg_num_page(page_dict,book_len):
  rslt = deepcopy(page_dict)
  for page,docs in page_dict.items():
    if page >= 0 : continue
    new_num = page+book_len
    del rslt[page]
    rslt[new_num] = docs
  return rslt

def insert_pages(doc_list,new_pages):
  new_pages = convert_neg_num_page(new_pages,len(doc_list))
  page_list = sorted(list(new_pages.keys()))[::-1]
  for page in page_list:
    if page >= len(doc_list) -1 : doc_list += new_pages[page]
    else : doc_list = doc_list[:page+1]+new_pages[page]+doc_list[page+1:]
  return doc_list

def get_table_page(num,doc_list,tab_mark,table):
    this_page = doc_list[num]['text']
    next_page = doc_list[num+1]['text'] if num+1 < len(doc_list) else ''
    both_page = this_page + next_page if next_page != '' else ''

    this_rslt,page0 = make_table_page(this_page,tab_mark,table)
    next_rslt,page1 = make_table_page(next_page,tab_mark,table)
    both_rslt,page2 = make_table_page(both_page,tab_mark,table)

    if this_rslt is not None : page_content,page = this_rslt, page0
    if next_rslt is not None : page_content,page,num = next_rslt, page1,num+1
    elif both_rslt is not None :
      page_content,page = both_rslt, page2[:len(this_page)+abs(len(both_page)-len(both_rslt))]
    else : page_content,page = None, this_page
    return page_content, page, num

def expand_pages(doc_list,new_pages):
  for page,tables in new_pages.items():
    content = doc_list[page]['text']+'\n'+'\n'.join(tables)
    doc_list[page]['text'] = content
  return doc_list

def insert_table_2_doc(doc_list,table_dict,tab_word='[[TABLE_{0}]]'):
  new_pages,cnt = defaultdict(list),0
  for num,table_info in table_dict.items():
    for tab_mark,table in table_info:
      table_page,page_adjst,page_num = get_table_page(num,doc_list,tab_mark,table)
      if table_page is not None:
        new_pages[page_num].append(table_page)
#        doc_list[page_num]['text'] = page_adjst
        cnt+=1
      else : new_pages[num].append(table+'\n'+tab_mark)

  doc_list = expand_pages(doc_list,new_pages)
  return doc_list, cnt/sum(map(len,table_dict.values()))

def insert_btwn_chr(sent,sep):
  c = list(add_escape(sent))
  d = c.copy()
  diff = (len(c)-len(sent))//2
  for i in range(2*diff+1,len(c)-diff*2+2)[::-1]:
    d.insert(i-1,sep)
  return ''.join(d)



In [16]:
import difflib

def union_strs(str0,str1):
    output_list = difflib.ndiff(str0, str1)
    return ''.join(map(lambda x : x[-1],output_list))

def pdf_2_chunck_w_table(file_path, tables, tab_word,chunk_size=256, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = pymupdf4llm.to_markdown(file_path,page_chunks=True,table_strategy='lines')
    doc,rate = insert_table_2_doc(doc,tables,tab_word)
    doc0 = '\n'.join(map(lambda x : x['text'],doc))
#    doc1 = pymupdf4llm.to_markdown(file_path)
#    docs = union_strs(doc0,doc1)
    md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_splitter.split_text(doc0)
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(md_chunks)
    print(file_path)
    print(f'table mark detect rate : {rate:.5f}')

    return chunks,rate

def make_chunk_dict_from_df(df, base_dir, table_dict, chunk_size=256):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    unique_paths = df['Source_path'].unique()
    chunk_dict = dict()
    err_tab_dict = dict()

    for file_path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        full_path = process_path(base_dir,file_path)
        full_path = processed_path_matcher(base_dir,full_path)
        pdf_title = os.path.basename(full_path)
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunk_dict[file_path]= pdf_2_chunck_w_table(full_path,table_dict[file_path],chunk_size)
    return chunk_dict

In [17]:
#앙상블
def process_pdfs_from_df(df, base_dir, table_dict, tab_word, chunk_size=256, model_path = "intfloat/multilingual-e5-small"):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    rate_dict=dict()

    for file_path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        full_path = process_path(base_dir,file_path)
        full_path = processed_path_matcher(base_dir,full_path)
        pdf_title = os.path.basename(full_path)
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks,rate =pdf_2_chunck_w_table(full_path,table_dict[file_path],tab_word,chunk_size)
        db = create_vector_db(chunks, model_path=model_path)

        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()
        retriever = EnsembleRetriever(
            retrievers=[kiwi_bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5],
            search_type="mmr",
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
        rate_dict[pdf_title] = rate
    return pdf_databases, rate_dict

## run codes

In [18]:
headers_to_split_on = [
    ("#","Header 1"),
    ("##","Header 2"),
    ("###","Header 3"),
]

tab_word = '!표{0}!'

In [20]:
train_df = pd.read_csv(f'{BASE_DIR}train.csv')
test_df = pd.read_csv(f'{BASE_DIR}test.csv')

#### move files to runtime

In [31]:
ls {BASE_DIR}

241008_csv_checker.ipynb             gemma2_financeQA-finetune/  test_source/
combined_train_aug_v3.5_editted.csv  processed/                  train.csv
combined_train_aug_v3.csv            sample_submission.csv       train_source/
combined_train_aug_v3_editted.csv    sub/                        Untitled0.ipynb
data/                                temp/
eval/                                test.csv


In [32]:
src_dirs = ['train_source/','test_source/']
file_path = ' '.join([os.path.join(BASE_DIR,sub) for sub in src_dirs])
temp_path = '/content/src/'
if not os.path.exists(temp_path) : os.makedirs(temp_path)

In [33]:
for sub in src_dirs:
  src_path = os.path.join(BASE_DIR,sub)
  dst_path = os.path.join(temp_path,sub)
  if not os.path.exists(dst_path) : os.makedirs(dst_path)
  !rsync -rvzh {src_path} {dst_path} --bwlimit 4096000000000000 --progress

sending incremental file list
1-1 2024 주요 재정통계 1권.pdf
         12.79M 100%   18.05MB/s    0:00:00 (xfr#1, to-chk=15/17)
2024 나라살림 예산개요.pdf
          7.63M 100%   18.82MB/s    0:00:00 (xfr#2, to-chk=14/17)
2024년도 성과계획서(총괄편).pdf
          5.66M 100%    4.14MB/s    0:00:01 (xfr#3, to-chk=13/17)
고용노동부_내일배움카드(일반).pdf
        143.29K 100%  164.44kB/s    0:00:00 (xfr#4, to-chk=12/17)
고용노동부_조기재취업수당.pdf
        166.37K 100%   63.71MB/s    0:00:00 (xfr#5, to-chk=11/17)
고용노동부_청년일자리창출지원.pdf
        135.51K 100%  200.81kB/s    0:00:00 (xfr#6, to-chk=10/17)
국토교통부_민간임대(융자).pdf
        116.94K 100%   80.28MB/s    0:00:00 (xfr#7, to-chk=9/17)
국토교통부_소규모주택정비사업.pdf
        305.15K 100%  460.59kB/s    0:00:00 (xfr#8, to-chk=8/17)
국토교통부_전세임대(융자).pdf
        136.00K 100%   98.45MB/s    0:00:00 (xfr#9, to-chk=7/17)
보건복지부_노인일자리 및 사회활동지원.pdf
 

In [34]:
## caution to unicode normalize
display('1-1 2024 주요 재정통계 1권.pdf' == normalize_path('1-1 2024 주요 재정통계 1권.pdf'))
display('1-1 2024 주요 재정통계 1권.pdf' == '1-1 2024 주요 재정통계 1권.pdf')

True

False

In [35]:
PROCESSEDDIR = os.path.join(BASE_DIR,'processed')
if not os.path.exists(PROCESSEDDIR) : os.makedirs(PROCESSEDDIR)
ERRORDIR = os.path.join(PROCESSEDDIR,'ERRORS')
if not os.path.exists(ERRORDIR) : os.makedirs(ERRORDIR)

In [37]:
reform_pdfs_from_df(train_df, temp_path,PROCESSEDDIR,'trn')
reform_pdfs_from_df(test_df, temp_path,PROCESSEDDIR,'tst');

Processing PDFs:   0%|          | 0/16 [00:00<?, ?it/s]

./train_source/1-1 2024 주요 재정통계 1권.pdf /content/src/
Processing 1-1 2024 주요 재정통계 1권.pdf...
/content/src/train_source/1-1 2024 주요 재정통계 1권.pdf /content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/processed/./train_source/1-1 2024 주요 재정통계 1권.pdf


  0%|          | 0/137 [00:00<?, ?it/s]

detected: (106.16761850585938, 262.17433237304687, 496.45875478515626, 664.8613426757812) 	 at (0.0, 0.0, 556.0, 754.0)  in depth 0
detected in p.2 :	 1  tables
detected: (181.28433298339843, 170.5902564453125, 486.7898705078125, 636.3243553710937) 	 at (0.0, 0.0, 556.0, 754.0)  in depth 0
Filling in gap at top of table
detected in p.5 :	 1  tables
detected: (154.67742990722655, 115.70205759277344, 484.81419301757813, 655.3303368164062) 	 at (0.0, 0.0, 556.0, 754.0)  in depth 0
detected in p.6 :	 1  tables
detected: (182.44290231933593, 113.56134104003907, 484.57386708984376, 623.3132469726562) 	 at (0.0, 0.0, 556.0, 754.0)  in depth 0
detected in p.7 :	 1  tables
detected: (197.99774240722655, 113.601853125, 486.1949913574219, 584.6992333007812) 	 at (0.0, 0.0, 556.0, 754.0)  in depth 0
detected in p.9 :	 1  tables
detected: (161.9510962890625, 103.98705362548829, 485.0072472167969, 643.39961171875) 	 at (0.0, 0.0, 556.0, 754.0)  in depth 0


<ipython-input-26-a0e14bef39ad>:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_temp = df.replace(to_replace=[None], value=0).fillna('').astype(str)


...

exception :  table does not sound
exception :  The identified boxes have significant overlap: 37.48% of area is overlapping (Max is 30.00%)
error :  (48.5129458984375, 373.93416477050783, 539.2971279541016, 772.9277431884766)
errs :  1
detected: (44.81023288574219, 79.10924411621093, 545.9518317301432, 111.94477199707032) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0
detected: (67.99853397216796, 405.66414523925783, 545.9518317301432, 540.5468227783203) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0
detected: (63.659094580078126, 184.6033935546875, 547.7153408447266, 368.3145470458984) 	 at (0.0, 111.94477199707032, 595.2760009765625, 405.66414523925783)  in depth 1
exception :  The identified boxes have significant overlap: 91.90% of area is overlapping (Max is 30.00%)
error :  (44.81023288574219, 79.10924411621093, 545.9518317301432, 111.94477199707032)
detected in p.2 :	 2  tables
detected: (48.8415337483724, 473.5321261962891, 540.858773461914

<ipython-input-26-a0e14bef39ad>:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_temp = df.replace(to_replace=[None], value=0).fillna('').astype(str)


detected in p.8 :	 3  tables
detected: (42.85623432006836, 75.56627536621093, 540.4986660400391, 108.40180324707032) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0
Filling in gap at top of table
exception :  The identified boxes have significant overlap: 131.72% of area is overlapping (Max is 30.00%)
error :  (42.85623432006836, 75.56627536621093, 540.4986660400391, 108.40180324707032)
detected: (54.57323337402344, 92.84123874511718, 546.6217739501953, 755.9483730712891) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0


<ipython-input-26-a0e14bef39ad>:41: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_temp = df.replace(to_replace=[None], value=0).fillna('').astype(str)


detected in p.10 :	 1  tables
detected: (48.77920986022949, 196.98920322265624, 540.8268346365793, 744.3124853384165) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0
detected in p.13 :	 1  tables
detected: (48.35423541870117, 237.19322849121093, 545.9959804931641, 270.0287563720703) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0
detected: (57.463042028808594, 74.41676975097656, 545.8958828369141, 180.10271572265626) 	 at (0.0, 0.0, 595.2760009765625, 237.19322849121093)  in depth 1
exception :  The identified boxes have significant overlap: 258.21% of area is overlapping (Max is 30.00%)
error :  (48.35423541870117, 237.19322849121093, 545.9959804931641, 270.0287563720703)
detected in p.14 :	 1  tables
detected: (47.82839084472656, 129.74704624023437, 453.05619167480467, 439.14811062011717) 	 at (0.0, 0.0, 595.2760009765625, 841.8900146484375)  in depth 0
detected in p.15 :	 1  tables
19 12 19


# Dataset Config

In [19]:
tab_ver = 'tab_v3.1'
model_dict = {
'large':"intfloat/multilingual-e5-large",
'base':"intfloat/multilingual-e5-base",
}

if tab_ver == 'tab_v0' : file_dir =BASE_DIR
else : file_dir = os.path.join(BASE_DIR,'processed',tab_ver)

model_option = 'large'
#model_option = 'base'
model_path = model_dict[model_option]
chunk_size = 512 #256
split_ver = 'split.2'

In [20]:
aug_type = "AugGPT"

In [21]:
aug_type= 'AugAEDA'

In [22]:
aug_type= 'GPTOnly'

In [23]:
aug_type= 'NoAug'

In [24]:
db_config = {
    'model' : model_option,
    'tab_process' : tab_ver,
    'aug' : aug_type,
    'chunck_size' : chunk_size,
    'split_ver' : split_ver
}

In [25]:
db_name = "{model}-ensemble-{tab_process}.{split_ver}-{chunck_size}".format(**db_config)

## Split train/valid

In [26]:
data_df = pd.read_csv(os.path.join(BASE_DIR,'train.csv'))
test_df = pd.read_csv(os.path.join(BASE_DIR,'test.csv'))

In [27]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

#fig,ax=plt.subplots()
#sns.histplot(data_df['Source'],ax=ax)
#a = len(data_df['Source'].unique())
#ax.plot(range(a),len(data_df)*0.2*np.ones(a),color='#112155')
#ax.plot(range(a),len(data_df)*0.225*np.ones(a),color='#115175')
#ax.plot(range(a),len(data_df)*0.175*np.ones(a),color='#115175')
#pass

In [28]:
import numpy as np

def check_split_possible(dist,ratio,err_ths):
  n_all = dist.sum()
  if dist.iloc[-1] > n_all * (ratio+err_ths) : return False
  cond = dist > n_all * (ratio+err_ths)
  if dist[cond].sum() > n_all*(1-ratio+err_ths) : return False
  else : return True

def split_by_files(data:pd.DataFrame,col,test_size,random_state,err_ths=0.025):
  if col not in data.columns : return False
  nprnd = np.random.RandomState(random_state)
  n_all = len(data)
  dist = data[col].value_counts().sort_values(ascending=False)
  if not check_split_possible(dist,test_size,err_ths): return False
  train_cond = dist > n_all*(test_size-err_ths)
  while dist[train_cond].sum() < n_all*(1-test_size-err_ths):
    cand = nprnd.choice(dist[~train_cond].index,1)[0]
    if dist[train_cond].sum()+dist.loc[cand] > n_all*(1-test_size+err_ths) : continue
    else : train_cond = (dist.index == cand) | train_cond
    # have to make hedge for infinite loop
  train_files,test_files = dist[train_cond].index, dist[~train_cond].index
  print(f'size : {dist[train_cond].sum()}, ratio : {dist[train_cond].sum()/n_all:.4f}, n_files : {len(train_files)}')
  print(f'size : {dist[~train_cond].sum()}, ratio : {dist[~train_cond].sum()/n_all:.4f}, n_files : {len(test_files)}')
  train_cond = data[col].isin(train_files)
  return data[train_cond], data[~train_cond]

In [29]:
from sklearn.model_selection import train_test_split

if split_ver == 'split.0' : train_df, valid_df = data_df, pd.DataFrame(columns=data_df.columns)
elif split_ver == 'split.1' :train_df,valid_df = train_test_split(data_df,test_size=0.2,stratify=data_df.Source,random_state=801)
elif split_ver == 'split.2' :train_df,valid_df = split_by_files(data_df,'Source',test_size=0.2,random_state=801)

size : 396, ratio : 0.7984, n_files : 11
size : 100, ratio : 0.2016, n_files : 5


## Apply Augmentation

In [30]:
ls {BASE_DIR}

241008_csv_checker.ipynb             gemma2_financeQA-finetune/  test_source/
combined_train_aug_v3.5_editted.csv  processed/                  train.csv
combined_train_aug_v3.csv            sample_submission.csv       train_source/
combined_train_aug_v3_editted.csv    sub/                        Untitled0.ipynb
data/                                temp/
eval/                                test.csv


In [31]:
refine_option = False
filter_option = False

file_config = {
    'refined' : refine_option,
    'filter' : filter_option,
}

In [32]:
if refine_option : aug_file,sep = 'combined_train_aug_v3.5_editted.csv','\tab'
else  : aug_file,sep = 'combined_train_aug_v3_editted.csv','|'
#aug_file = 'combined_train_aug_v3.csv'
aug_path = os.path.join(BASE_DIR,aug_file)

In [33]:
ques_dict={
    'NoAug' : 'Question',
    'AugGPT' : 'Question_aug_GPT',
    'AugAEDA' : 'AEDA_Question',
    'GPTOnly' : 'Question_aug_GPT',
}
ans_dict = {
    'NoAug' : 'Answer',
    'AugGPT' : 'Answer',
    'AugAEDA' : 'Answer',
    'GPTOnly' : 'Answer',
}

In [34]:
key_col = 'SAMPLE_ID'
info_col = ['Source', 'Source_path']
ques_base = ques_dict['NoAug']
ans_base = ans_dict['NoAug']
ques_col = ques_dict[aug_type]
ans_col = ans_dict[aug_type]

In [35]:
filter_list = ['TRAIN_451', 'TRAIN_452', 'TRAIN_453', 'TRAIN_454', 'TRAIN_455', 'TRAIN_456']

In [36]:
aug_df = pd.read_csv(aug_path,sep=sep)
aug_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   SAMPLE_ID         496 non-null    object
 1   Source            496 non-null    object
 2   Source_path       496 non-null    object
 3   Question          496 non-null    object
 4   Answer            496 non-null    object
 5   Question_aug_GPT  496 non-null    object
 6   Answer_aug_GPT    496 non-null    object
 7   AEDA_Question     496 non-null    object
 8   AEDA_Answer       496 non-null    object
dtypes: object(9)
memory usage: 35.0+ KB


In [37]:
import numpy as np
train_id = train_df[key_col].values
#print(pd.Series(filter_list).isin(train_id))
cond = (aug_df[key_col].isin(train_id))
if filter_option : cond = cond & (~(aug_df[key_col].isin(filter_list)))
display(aug_df.columns), len(train_id), np.sum(cond)

Index(['SAMPLE_ID', 'Source', 'Source_path', 'Question', 'Answer',
       'Question_aug_GPT', 'Answer_aug_GPT', 'AEDA_Question', 'AEDA_Answer'],
      dtype='object')

(None, 396, 396)

In [38]:
col_list = [key_col]+info_col+[ques_base,ans_base]
train_adjst = aug_df.loc[cond,col_list]
train_df= train_adjst.rename(columns = {ques_col : "Question", ans_col : "Answer"})

In [39]:
if aug_type != 'NoAug':
  col_list = [key_col]+info_col+[ques_col,ans_col]
  aug_train = aug_df.loc[cond,col_list]
  aug_train = aug_train.rename(columns = {ques_col : "Question", ans_col : "Answer"})
  if 'Only' in aug_type : train_augged=aug_train
  else : train_augged= pd.concat([train_df,aug_train])
  train_augged.info()

In [40]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 50 to 478
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   SAMPLE_ID    396 non-null    object
 1   Source       396 non-null    object
 2   Source_path  396 non-null    object
 3   Question     396 non-null    object
 4   Answer       396 non-null    object
dtypes: object(5)
memory usage: 18.6+ KB


In [41]:
valid_id = valid_df[key_col].values
cond = (aug_df[key_col].isin(valid_id))
if filter_option : cond = cond & (~(aug_df[key_col].isin(filter_list)))
col_list = [key_col]+info_col+[ques_base,ans_base]
valid_adjst = aug_df.loc[cond,col_list]
valid_df= valid_adjst.rename(columns = {ques_col : "Question", ans_col : "Answer"})

In [42]:
free_cuda()

freed :  30
freed :  0


# DB 생성

In [43]:
temp_path = '/content/processed/src/'
file_dir, os.listdir(file_dir)

('/content/drive/MyDrive/kdt-EST-AI/project/dacon_fis/src/processed/tab_v3.1',
 ['train_source', 'tables', 'test_source'])

In [44]:
src_dirs = list(filter(lambda x : x != 'pdf_db',os.listdir(file_dir)))
file_path = ' '.join([os.path.join(file_dir,sub) for sub in src_dirs])
if not os.path.exists(temp_path) : os.makedirs(temp_path)

In [49]:
!rsync -rvzh {file_path} {temp_path} --bwlimit 4096000000000000 --progress

sending incremental file list
tables/
tables/tab_trn.pkl
          3.21M 100%  168.29MB/s    0:00:00 (xfr#1, to-chk=76/80)
tables/tab_tst.pkl
        250.27K 100%  253.27kB/s    0:00:00 (xfr#2, to-chk=75/80)
test_source/
test_source/err_국토교통부_행복주택출자.pkl
          1.09K 100%    0.00kB/s    0:00:00 (xfr#3, to-chk=74/80)
test_source/err_보건복지부_노인장기요양보험 사업운영.pkl
              5 100%    0.01kB/s    0:00:00 (xfr#4, to-chk=73/80)
test_source/err_보건복지부_부모급여(영아수당) 지원.pkl
              5 100%    0.00kB/s    0:00:00 (xfr#5, to-chk=72/80)
test_source/err_산업통상자원부_에너지바우처.pkl
              5 100%    0.00kB/s    0:00:01 (xfr#6, to-chk=71/80)
test_source/err_중소벤처기업부_혁신창업사업화자금(융자).pkl
              5 100%    0.00kB/s    0:00:00 (xfr#7, to-chk=70/80)
test_source/err_「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pkl
              5 100%    0.00kB/s    0:00:01 (xfr#8, to-chk=69/80)
te

In [50]:
#use it when table ver < 2
def process_pdf(file_path, chunk_size=256, chunk_overlap=32):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = pymupdf4llm.to_markdown(file_path)

    headers_to_split_on = [
        ("#","Header 1"),
        ("##","Header 2"),
        ("###","Header 3"),
    ]

    md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
    md_chunks = md_splitter.split_text(doc)

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunks = splitter.split_documents(md_chunks)

    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db


#앙상블
def process_pdfs_from_dataframe(df, base_dir, chunk_size=256, model_path = "intfloat/multilingual-e5-small"):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for file_path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        full_path = process_path(base_dir,file_path)
        full_path = processed_path_matcher(base_dir,full_path)
        pdf_title = os.path.basename(full_path)
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path,chunk_size)
        db = create_vector_db(chunks, model_path=model_path)

        kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
        faiss_retriever = db.as_retriever()
        retriever = EnsembleRetriever(
            retrievers=[kiwi_bm25_retriever, faiss_retriever],
            weights=[0.5, 0.5],
            search_type="mmr",
        )

        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases


In [45]:
if float(tab_ver[5:]) >= 2:
  pkl_dir = os.path.join(temp_path,'tables')
  tab_dict_trn = load_pkl(os.path.join(pkl_dir,'tab_trn.pkl'))
  tab_dict_tst= load_pkl(os.path.join(pkl_dir,'tab_tst.pkl'))

In [46]:
tab_word, tab_ver

('!표{0}!', 'tab_v3.1')

In [53]:
if float(tab_ver[5:]) >= 2:
  train_db, detect_rate = process_pdfs_from_df(train_df, temp_path, tab_dict_trn, tab_word, chunk_size=chunk_size, model_path=model_path)
else : train_db = process_pdfs_from_dataframe(train_df, temp_path, chunk_size=chunk_size, model_path=model_path)

Processing PDFs:   0%|          | 0/11 [00:00<?, ?it/s]

Processing 2024 나라살림 예산개요.pdf...
Processing /content/processed/src/train_source/2024 나라살림 예산개요.pdf...
[                                        ] (0/314)[                                        ] (  1/314)[                                        ] (  2/314)[                                        ] (  3/314)[                                        ] (  4/314)[                                        ] (  5/314)

<ipython-input-50-63aeb89c10a1>:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Processing 재정통계해설.pdf...
Processing /content/processed/src/train_source/재정통계해설.pdf...
[                                        ] (0/164)[                                        ] (  1/164)[                                        ] (  2/164)[                                        ] (  3/164)[                                        ] (  4/164)=[=                                       ] (  5/164)

In [54]:
free_cuda()

freed :  20
freed :  0


In [55]:
if split_ver == 'split.2':
  if float(tab_ver[5:]) >= 2:
    valid_db, detect_rate = process_pdfs_from_df(valid_df, temp_path, tab_dict_trn, tab_word, chunk_size=chunk_size, model_path=model_path)
  else : valid_db = process_pdfs_from_dataframe(valid_df, temp_path, chunk_size=chunk_size, model_path=model_path)
else : valid_db = train_db

Processing PDFs:   0%|          | 0/5 [00:00<?, ?it/s]

Processing 1-1 2024 주요 재정통계 1권.pdf...
Processing /content/processed/src/train_source/1-1 2024 주요 재정통계 1권.pdf...
[                                        ] (0/137)[                                        ] (  1/137)[                                        ] (  2/137)[                                        ] (  3/137)=[=                                       ] (  4/137)[=                                       ] (  5/137)

In [47]:
aug_type

'NoAug'

In [48]:
if float(tab_ver[5:]) >= 2:
  test_db, detect_rate = process_pdfs_from_df(test_df, temp_path, tab_dict_tst, tab_word[1:-1], chunk_size=chunk_size, model_path=model_path)
else : test_db  = process_pdfs_from_dataframe(test_df, temp_path, chunk_size=chunk_size, model_path=model_path)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자).pdf...
Processing /content/processed/src/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf...
[                                        ] (0/3)=============[=============                           ] (1/3)=============[==========================              ] (2/3)==============[========================================] (3/3)]
/content/processed/src/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
table mark detect rate : 0.60000


<ipython-input-12-42f6f99729c4>:53: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Processing 보건복지부_부모급여(영아수당) 지원.pdf...
Processing /content/processed/src/test_source/보건복지부_부모급여(영아수당) 지원.pdf...
[                                        ] (0/3)=============[=============                           ] (1/3)=============[==========================              ] (2/3)==============[========================================] (3/3)]
/content/processed/src/test_source/보건복지부_부모급여(영아수당) 지원.pdf
table mark detect rate : 0.60000
Processing 보건복지부_노인장기요양보험 사업운영.pdf...
Processing /content/processed/src/test_source/보건복지부_노인장기요양보험 사업운영.pdf...
[                                        ] (0/4)

In [ ]:
file_dir

In [ ]:
db_name

In [ ]:
#db_path = os.path.join('/content','pdf_db')
db_path = os.path.join(file_dir,'pdf_db')
#save_pkl(db_path, f'{db_name}_train.dat',train_db)
#save_pkl(db_path, f'{db_name}_test.dat',test_db)

In [ ]:
os.listdir(db_path)

In [ ]:
train_db_name = 'large-ensemble-tab_v1.7-256_train.dat'
test_db_name = 'large-ensemble-tab_v1.7-256_test.dat'
train_db_path = os.path.join(db_path,train_db_name)
test_db_path = os.path.join(db_path,test_db_name)

In [ ]:
file_path = ' '.join([#train_db_path,
            test_db_path
                      ])
temp_path = '/content/pdf_db'
if not os.path.exists(temp_path) : os.makedirs(temp_path)

In [ ]:
!rsync -vzh {file_path} {temp_path} --bwlimit 4096000000000000 --progress

In [ ]:
#train_db = load_pkl(os.path.join(temp_path,train_db_name))
test_db = load_pkl(os.path.join(temp_path,test_db_name))

# Create Dataset

In [49]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

def make_dataset(df, pdf_databases):
    dataset = dict()
    dataset['context'] = list()
    dataset['question'] = list()
    dataset['answer'] = list()
    normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Making"):
        # 소스 문자열 정규화
        source = normalize_string(row['Source'])+'.pdf'
        question = row['Question']
        dataset['question'].append(question)
        if 'Answer' in df.columns:
          dataset['answer'].append(row['Answer'])
        else: dataset['answer'].append('')

        # 정규화된 키로 데이터베이스 검색
        retriever = normalized_keys[source]['retriever']
        context = format_docs(retriever.invoke(question))
        dataset['context'].append(context)
    return dataset


# Dataset

In [50]:
if aug_type != 'NoAug':
  train_df = train_augged

In [51]:
if (refine_option, filter_option) == (False, False) : prefix = ''
elif (refine_option, filter_option) == (False, True) : prefix = 'filtered'
elif (refine_option, filter_option) == (True, False) : prefix = 'refined0'
elif (refine_option, filter_option) == (True, True) : prefix = 'refined'

In [52]:
dataset_name = "kdt3/DACON-QA-{model}-ensemble-{tab_process}.{split_ver}-{prefix}{aug}-{chunck_size}".format(prefix=prefix,**db_config)
train_name = "kdt3/DACON-QA-{model}-ensemble-{tab_process}.{split_ver}-{prefix}{aug}-{chunck_size}".format(prefix=prefix,**db_config)
#fname = "gemma2_large_ensemble_markdown_256_5epoch_reprocessed_result.csv"

push_url = dataset_name
push_url

'kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512'

In [ ]:
## 만약 데이터셋을 분할해서 업로드해줘야할 경우 합치는 방법 참조 코드
from datasets import load_dataset, concatenate_datasets
from datasets import Dataset

train_dataset = load_dataset(dataset_name)['train']

train_dataset = concatenate_datasets([train_dataset, Dataset.from_dict(make_dataset(train_df.iloc[296:], train_db))])
train_dataset.push_to_hub(dataset_name, private=True, split='train')


## Train 데이터 생성 & 업로드

In [60]:
from datasets import Dataset
train_dataset = make_dataset(train_df, train_db)
train_dataset = Dataset.from_dict(train_dataset)
train_dataset.push_to_hub(push_url, private=True, split='train')


Making:   0%|          | 0/396 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512/commit/0ae152851d854fffa6c71e373e564c2133bab448', commit_message='Upload dataset', commit_description='', oid='0ae152851d854fffa6c71e373e564c2133bab448', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512'), pr_revision=None, pr_num=None)

## Valid 데이터 생성 & 업로드

In [61]:
from datasets import Dataset
valid_dataset = make_dataset(valid_df, valid_db)
valid_dataset = Dataset.from_dict(valid_dataset)
valid_dataset.push_to_hub(push_url, private=True, split='valid')

Making:   0%|          | 0/100 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/347 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512/commit/738c5c0364d5c23148ab98f61c5d1da9f9f2f510', commit_message='Upload dataset', commit_description='', oid='738c5c0364d5c23148ab98f61c5d1da9f9f2f510', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512'), pr_revision=None, pr_num=None)

## Test 데이터 생성 & 업로드

In [53]:
from datasets import Dataset
test_dataset = make_dataset(test_df, test_db)
test_dataset = Dataset.from_dict(test_dataset)
test_dataset.push_to_hub(push_url, private=True, split='test')

Making:   0%|          | 0/98 [00:00<?, ?it/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/447 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512/commit/23b951ff8468e25232cd3005ca5cad78a254ce26', commit_message='Upload dataset', commit_description='', oid='23b951ff8468e25232cd3005ca5cad78a254ce26', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512', endpoint='https://huggingface.co', repo_type='dataset', repo_id='kdt3/DACON-QA-large-ensemble-tab_v3.1.split.2-NoAug-512'), pr_revision=None, pr_num=None)

# debug
-

In [ ]:
def get_tables_from_pdf(full_path,tab_word='[[TABLE_{0}]]'):
    pdf = pymupdf.open(full_path)
    chunks, tables_dict, cnt= list(), defaultdict(list),0
    err_dict = dict()
    if pdf is None : return None, tables_dict
    for pnum, page in enumerate(tqdm(pdf)):
      if pnum < 7 : continue
      return page
      page_area = tuple(page.mediabox)
      try:
        tab_rslt = search_page(page,page_area,get_ths(page_area))
        bboxes, errors = tab_rslt['tabs'],tab_rslt['errs']
        tables,doc =list(), PyMuPDFPage(page)
        formatter = define_formatter()
        for box in bboxes:
          table = make_table(box['bbox'],doc,page_area,formatter,get_ths(page_area))
          if table is None :
            print('error : ',box['bbox'])
            continue
          if type(table) is not dict : errors.append(table)
          else :
            table['depth'] = box['depth']
            tables.append(table)

      except:
        save_pkl(ERRORDIR,'page_doc_error,pkl',[full_path,pnum,page_area])
        raise Exception()
      if len(errors)>0 :
        err_dict[pnum] = errors
        print('errs : ',len(errors))
      if len(tables) == 0 : continue
      print(f'detected in p.{pnum} :\t',len(tables),' tables')
      tables = sorted(tables,key=lambda x : (x['bbox'][0],x['bbox'][1]))
      for idx,tab in enumerate(tables):
        tab_mark = tab_word.format(cnt+idx)
        table_md = clean_table(tab['content'].to_markdown(index=False))
        tables_dict[pnum].append((tab_mark,table_md + f"\n{tab['caption']}"))

        try :
          area = tab['bbox']
          page.add_redact_annot(area)
          page.apply_redactions()
          page.draw_rect(area,color=(.0,0,0),fill=(.99,.99,.99))
          rc = page.insert_htmlbox(area,tab_mark,scale_low=0)
        except :
          print(page.mediabox)
          print(page.cropbox)
          print(tab['bbox'])
          display(tab['content'])
          print(table_md)

      cnt+=len(tables)

    print(cnt, len(tables_dict), sum([len(tables) for tables in tables_dict.values()]))
    return pdf, tables_dict,err_dict




In [ ]:
s = '''
index,,
0,"02 재정지출","104"
1,"1. 국가별","104"
2,"2. 기능별 재정지출","106"
3,"3. 공공 사회 복지 지출","107"
4,"03 재정건전성","108"
5,"1. 재정수지","108"
6,"(1) 재정수지","108"
7,"(2) 기초 재정수지","109"
8,"2. 부채","110"
9,"(1) 부채","110"
10,"(2) 1인당 부채","111"
11,"주요재정통계","113"
12,"01 총수입･국세수입","116"
13,"02 조세부담률 및 국민부담률","118"
14,"03 총지출","120"
15,"04 분야별 재정지출","122"
16,"05 의무지출･재량지출 통합재정수지･국가채무","124"
17,"06","126"
18,"07 지방재정조정","126"
'''
import sys
if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

content = StringIO(s)
df = pd.read_table(content, sep=",",index_col=0)
display(df.info())
check_table_df_soundness(df)

In [ ]:
df,base_path,save_dir = train_df, temp_path,PROCESSEDDIR
path = './train_source/1-1 2024 주요 재정통계 1권.pdf'
tab_rslt,err_rslt=extract_table_and_pdf(path,base_path,save_dir)

In [ ]:
base_dir = temp_path
file_path = './train_source/1-1 2024 주요 재정통계 1권.pdf'
full_path = process_path(base_dir,file_path)
full_path = processed_path_matcher(base_dir,full_path)

page = get_tables_from_pdf(full_path)

In [ ]:
def search_page(page,area,ths=0,depth=0):
  if not larger_v_ths(area,ths) : return defaultdict(list)
  if depth >=10 : raise Exception(depth)
#  print('depth : ',depth,'area : ',area)
  page.set_cropbox(area) #area : page에서 절대적 위치. cropbox를 하게 되면 상대적 위치로 바뀜
  rslt,searched = defaultdict(list),list()
#  set_trace()
  detected = find_tables(page,get_page_size(area),ths)
#  return detected
  if len(detected)==0 : return rslt
  for target in detected:
    if target is None : continue
    if type(target) is not dict : rslt['errs'].append(target)
    rect = infer_bbox_pos(area,target['bbox'])
    bbox = {'bbox':rect,'depth':depth}
    rslt['tabs'].append(bbox)
    print('detected:',rect,'\t at',area,f' in depth {depth}')

    left_area = get_area(rect,area,True)
    right_area = get_area(rect,area,False)
    if depth > 5 : print(f'left {left_area}\tright{right_area}')
    left = search_page(page,left_area,ths,depth+1)
    right = search_page(page,right_area,ths,depth+1)
    searched.append((area[0],rect[1],area[2],rect[3]))
    if depth > 5 : print(searched)
    extend_list_dict(rslt,left)
    extend_list_dict(rslt,right)

  if len(rslt)==0 : return rslt
  searched = organize_box(searched,area,ths)
  blanks = get_blank(searched,area,ths)
  if depth >5 : print(f'in {area}\n\t',blanks)
  for row in blanks:
    detected = search_page(page,row,ths,depth+1)
    extend_list_dict(rslt,detected)

#  rslt= organize_box(rslt,area,ths) : can't apply directly like this
#  print('in search page, err : ',len(rslt['errs']))
  page.set_cropbox(area)
  return rslt

In [ ]:
search_page(page,page.mediabox)

In [ ]:
#check_table_df_soundness(detected[0])
df_temp = detected[0].replace(to_replace=[None], value=0).astype(str)
df_temp = df_temp.fillna('').astype(str)
val = '0'
cols =range(len(df_temp.columns))
df_temp.columns =cols
cond = df_temp == val
for col in cols:
  display(np.sum(cond[col]))
#target = list(filter(lambda col : np.sum(cond[col]) != len(cond),cols))
#erase_constant_rowcol(df_temp,'0')

In [ ]:
ls {PROCESSEDDIR}/train_source

In [ ]:
errs = load_pkl(os.path.join(PROCESSEDDIR,'train_source','err_1-1 2024 주요 재정통계 1권.pkl'))
len(errs)

In [ ]:
errs[6]

In [ ]:
def check_table_df_soundness(df,ths=0.5):
  if len(df) < 1 : return False
  df_temp = df.replace(to_replace=None, value=0).astype(str)
  if maybe_numeric_table(df_temp) : return True
  df_temp = df.replace(to_replace=None, value='PD_NONE')
  cols = list(df.columns.astype(np.string_))
  cols = list(map(lambda x : '' if x is None else str(x),cols))
  cols = list(map(lambda x : re.sub(r'[\s]*','',x),cols))
  null_col = list(filter(lambda x: len(x)<1,cols))
  if len(null_col) > len(cols) * ths : return False
  if np.sum(df_temp=='PD_NONE') > len(df)*len(cols)*ths : return False
  return True


def maybe_numeric_table(df,ths=0.35):
  df_temp = df.fillna('').astype(str)
  df_temp = df.replace(r'(?:(\d+?)),(\d+?)',r'\1\2',regex=True)
  df_temp = df_temp[list(df_temp.columns)].apply(pd.to_numeric,errors='coerce')
  rslt = ~df_temp.isna()
  rowwise = rslt.apply(sum,axis=1)
  colwise = rslt.apply(sum,axis=0)
  if np.sum(rowwise == len(rslt)) > 0 : return True
  if np.sum(colwise== len(rslt.columns)) > 0 : return True
  if np.sum(rslt.values) > len(rslt)*len(rslt.columns)*ths : return True
  return False

In [ ]:
bad = [{"index":0,"":"2018년부터 발간한 ｢주요 재정통계｣는 디지털예산회계시스템의 재정정보를"},{"index":1,"":"계에 따른 시계열 통계로 구성･제공하고 있습니다. 아울러 중앙-지방정부 등 여러"},{"index":2,"":"산재되어 있는 재정통계를 수록하여 재정분석의 기초자료로 활용될 수 있도록"},{"index":3,"":"다. 또한 단순 정보전달에 그치지 않고, 일반 국민도 쉽게 이해할 수 있도록 재정통계의 가독성을 높이고자 노력하였습니다."},{"index":4,"":"특히, 올해부터는 한권으로 제공되던 ｢주요 재정통계｣를 Ⅰ, Ⅱ 권으로 분권하여,"},{"index":5,"":"편의성을 제고하고자 하였습니다. <제Ⅰ권>에서 우리나라의 재정체계, 주요 재정지표"},{"index":6,"":"재정 전반에 대한 이해를 돕기 위해 주요 재정 제도의 설명과 함께 관련 통계를 수록하였고,"},{"index":7,"":"OECD, IMF 회원국 간 재정 동향을 비교･분석할 수 있는 통계로 구성하였습니다."},{"index":8,"":"<제Ⅱ권>에서는 예산체계에 따른 16대 분야별 재정 구조와 추이, 사업유형별 주요사업 정보를 담아 각 분야별 재정지출에 대한 이해도를 높일 수 있도록 구성하였습니다. 이와 함께, 부록에서는 국내 핵심 재정 통계를 선정하여 10년 이상의 장기 시계열"},{"index":9,"":"표를 추가 제공하였습니다."},{"index":10,"":"한국재정정보원은 ｢2024 주요 재정통계｣가 국민들의 재정에 대한 이해도를"},{"index":11,"":"재정당국에게는 재정정책 수립의 기초자료로 활용되길 바랍니다. 앞으로 시의적절함과"},{"index":12,"":"동시에 정합성을 담보한 다양한 재정자료를 지속적으로 제공할 수 있도록 최선을 습니다."},{"index":13,"":"2024년"}]
df = pd.DataFrame(bad)
df_temp = df.replace(to_replace=[None], value=0).astype(str)
df_temp = df_temp.fillna('').astype(str)
df_temp = df.replace(r'(?:(\d+?)),(\d+?)',r'\1\2',regex=True)
df_temp = df_temp[list(df_temp.columns)].apply(pd.to_numeric,errors='coerce')
rslt = ~df_temp.isna()
rowwise = rslt.apply(sum,axis=1)
colwise = rslt.apply(sum,axis=0)

In [ ]:
np.sum(rowwise)

In [ ]:
cols

In [ ]:
rslt

In [ ]:
np.sum(rslt)

In [ ]:
df_temp = df.replace(to_replace=[None], value=0).astype(str)
if maybe_numeric_table(df_temp) : print(True)

In [ ]:
file_path